In [1]:
from nltk.corpus import wordnet as wn
from copy import deepcopy

import random

In [2]:
terms_vocab = "/home/johannes/thesis_code/data_experimentation/trucks/term_vocab.txt"
full_vocab = "/home/johannes/thesis_code/data_experimentation/vocab_concat/vocabulary.txt"

In [3]:
tf = open(terms_vocab,'r')
terms_list = [w.strip('\n') for w in tf]
tf.close()
print(len(terms_list))

72184


In [4]:
def check_synsets(terms):
    '''
    Returns a list with all terms that have a wordnet synset
    '''
    return [w for w in terms if wn.synsets(w.replace(' ','_'))]

def select_synsets(terms_with_synsets):
    '''
    Returns a dictionary with first available synset with
    a noun POS tag
    '''
    synset_dict = {}
    for t in terms_with_synsets:
        ss = wn.synsets(t.replace(' ','_'))
        for s in ss:
            if '.n' in s.name():
                synset_dict[t] = s
                break
    return synset_dict
        
def make_hypernym_dict(synset_dict,num_levels=2):
    '''
    Returns a dictionary of terms associated with
    its hypernym synsets, traverses num_levels levels
    in the wordnet hierarchy
    '''
    hypernym_dict = {}
    for term in synset_dict:
        cur_level = 0
        next_hypernyms = synset_dict[term].hypernyms()
        
        hypernym_dict[term] = next_hypernyms
        cur_level += 1
        while cur_level < num_levels:
            _next_hypernyms = []
            for h in next_hypernyms:
                _next_hypernyms += h.hypernyms()
            hypernym_dict[term] += _next_hypernyms
            next_hypernyms = _next_hypernyms
            cur_level += 1
            
    return hypernym_dict

def keep_vocab_hypernyms(hypernym_dict, full_vocab_list):
    '''
    Removes hypernyms that do not exist in the
    vocabulary
    '''
    num_found = 0
    processed_terms = 0
    new_dict = {}
    for term in hypernym_dict:
        hypernyms_to_keep = []
        for h in hypernym_dict[term]:
            hypernym_text = h.name().split('.n')[0].replace('_',' ') 
            if hypernym_text in full_vocab_list:
                hypernyms_to_keep += [hypernym_text]
        if hypernyms_to_keep:
            new_dict[term] = hypernyms_to_keep
        processed_terms += 1
        if processed_terms%100 == 0:
            print("Processed ", str(processed_terms), " terms")
    return new_dict
        
    
def sample_terms(hypernym_dict, num_samples=100):
    '''
    Returns a random sample of num_samples terms
    '''
    all_terms = [t for t in hypernym_dict]
    return random.sample(all_terms, num_samples)

def write_training_data(queries, hypernym_dict, query_file, gold_file):
    '''
    Writes a query file and a gold file in accordance with the 
    SemEval-2018 Task 9 standard
    '''
    qf = open(query_file,'w+')
    gf = open(gold_file,'w+')
    for q in queries:
        qf.write(q)
        qf.write('\n')
        unique_hypernyms = list(set(hypernym_dict[q]))
        for i in range(len(unique_hypernyms)):
            gf.write(unique_hypernyms[i])
            if not i == (len(unique_hypernyms)-1):
                gf.write('\t')
        gf.write('\n')
    qf.close()
    gf.close()
    
def sanity_check(hypernym_dict, full_vocab_list):
    '''
    Performs a sanity check to make sure that no 
    out-of-vocabulary terms remain
    '''
    for term in hypernym_dict:
        if term not in full_vocab_list:
            print("Found out of vocab term: ", term ," something went wrong!")
            return
        for h in hypernym_dict[term]:
            if h not in full_vocab_list:
                print("Found out of vocab term: ", h , " something went wrong!")
                return
    print("All good!\n")

In [5]:
terms_with_synsets = check_synsets(terms_list)
print(len(terms_with_synsets))

6953


In [7]:
term_synset_dict = select_synsets(terms_with_synsets)
print_num =2
for key in term_synset_dict:
    print(key, term_synset_dict[key], term_synset_dict[key].hypernyms())
    print_num -= 1
    if not print_num:
        break

ccs Synset('milliliter.n.01') [Synset('metric_capacity_unit.n.01')]
biomass Synset('biomass.n.01') [Synset('fuel.n.01')]


In [8]:
hypernym_dict = make_hypernym_dict(term_synset_dict,num_levels=3)

In [9]:
print_num = 2
for t in hypernym_dict:
    print(t,hypernym_dict[t])
    print_num -= 1
    if not print_num:
        break

ccs [Synset('metric_capacity_unit.n.01'), Synset('metric_unit.n.01'), Synset('volume_unit.n.01'), Synset('unit_of_measurement.n.01'), Synset('unit_of_measurement.n.01')]
modernisation [Synset('improvement.n.02'), Synset('change_of_state.n.01'), Synset('change.n.03')]


In [10]:
fvf = open(full_vocab,'r')
full_vocab_list = [w.strip('\n') for w in fvf]
fvf.close()
print(len(full_vocab_list))

281828


In [11]:
ready_hypernyms = keep_vocab_hypernyms(hypernym_dict, full_vocab_list)

Processed  100  terms
Processed  200  terms
Processed  300  terms
Processed  400  terms
Processed  500  terms
Processed  600  terms
Processed  700  terms
Processed  800  terms
Processed  900  terms
Processed  1000  terms
Processed  1100  terms
Processed  1200  terms
Processed  1300  terms
Processed  1400  terms
Processed  1500  terms
Processed  1600  terms
Processed  1700  terms
Processed  1800  terms
Processed  1900  terms
Processed  2000  terms
Processed  2100  terms
Processed  2200  terms
Processed  2300  terms
Processed  2400  terms
Processed  2500  terms
Processed  2600  terms
Processed  2700  terms
Processed  2800  terms
Processed  2900  terms
Processed  3000  terms
Processed  3100  terms
Processed  3200  terms
Processed  3300  terms
Processed  3400  terms
Processed  3500  terms
Processed  3600  terms
Processed  3700  terms
Processed  3800  terms
Processed  3900  terms
Processed  4000  terms
Processed  4100  terms
Processed  4200  terms
Processed  4300  terms
Processed  4400  ter

In [12]:
print_num = 5
for t in ready_hypernyms:
    print(t, ready_hypernyms[t])
    print_num -= 1
    if not print_num:
        break

ccs ['metric unit', 'volume unit']
biomass ['fuel']
bathrobe ['robe', 'clothing']
pitch accent ['stress', 'prosody', 'manner of speaking']
local ['public transport', 'conveyance']


In [13]:
queries = sample_terms(ready_hypernyms,300)

In [14]:
print(len(queries))

300


In [15]:
query_file = "/home/johannes/thesis_code/data_experimentation/new_training_data/queries.txt"
gold_file = "/home/johannes/thesis_code/data_experimentation/new_training_data/gold.txt"
write_training_data(queries, ready_hypernyms, query_file, gold_file)

In [16]:
sanity_check(ready_hypernyms, full_vocab_list)

All good!

